In [59]:
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    PER,
    NamesExtractor,

    Doc
)
import json
import markup
import natasha

from bs4 import BeautifulSoup as Soup
import chardet
import unicodedata
import re

In [60]:
def get_raw_text(path_to_file):
    with open(path_to_file, 'rb') as f:
        rtext = f.read()
        rtext_unicode = rtext.decode(chardet.detect(rtext)["encoding"]).encode("utf8").decode("utf8")
        text = unicodedata.normalize("NFKD", rtext_unicode)
        soup = Soup(text[text.find('<body>'):], features="lxml")

    without_strong = []
    for offer in soup.find_all('p'):
        if '<strong>' not in str(offer) and '<emphasis>' not in str(offer):
            without_strong.append(offer.get_text(strip=True))
    text = ' '.join(without_strong[2:])
    return text


In [61]:
text = get_raw_text('Данные/Не обработанные/Белый/belyj_a-text_0040.fb2')

In [64]:
def get_doc(text):
    doc = Doc(text)

    segmenter = Segmenter()
    doc.segment(segmenter)

    emb = NewsEmbedding()
    tagger = NewsMorphTagger(emb)
    doc.tag_morph(tagger)

    morph_vocab = MorphVocab()
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    return doc

In [65]:
# функция get_doc от get_raw достает doc-и для других функицй
doc = get_doc(get_raw_text('Данные/Не обработанные/Белый/belyj_a-text_0040.fb2'))

In [66]:
doc.tokens

[DocToken(stop=8, text='Источник', pos='NOUN', feats=<Inan,Nom,Masc,Sing>, lemma='источник'),
 DocToken(start=8, stop=9, text=':', pos='PUNCT', lemma=':'),
 DocToken(start=10, stop=17, text='Андрей', pos='PROPN', feats=<Anim,Nom,Masc,Sing>, lemma='андрей'),
 DocToken(start=18, stop=24, text='Белый', pos='PROPN', feats=<Anim,Nom,Masc,Sing>, lemma='белый'),
 DocToken(start=24, stop=25, text=';', pos='PUNCT', lemma=';'),
 DocToken(start=26, stop=27, text='"', pos='PUNCT', lemma='"'),
 DocToken(start=27, stop=36, text='Петербург', pos='PROPN', feats=<Inan,Nom,Masc,Sing>, lemma='петербург'),
 DocToken(start=36, stop=37, text='"', pos='PUNCT', lemma='"'),
 DocToken(start=37, stop=38, text=',', pos='PUNCT', lemma=','),
 DocToken(start=39, stop=44, text='Роман', pos='PROPN', feats=<Anim,Nom,Masc,Sing>, lemma='роман'),
 DocToken(start=44, stop=45, text='.', pos='PUNCT', lemma='.'),
 DocToken(start=46, stop=52, text='Изд-во', pos='PROPN', feats=<Anim,Nom,Masc,Sing>, lemma='изд-во'),
 DocToke

In [101]:
import re
def has_cyrillic(text):
    return bool(re.search('[а-яА-Я]', text))


def get_sents(doc):
    sents_list = []
    for sent in doc.sents:
        sents_list.append(sent.text)

    return sents_list

def get_words(doc):
    only_words = []
    for token in doc.tokens:
        if has_cyrillic(token.text):
            only_words.append(token.text)
    return only_words

def get_lemma_words(doc):
    #lemma_words = {_.text: _.lemma for _ in doc.tokens}
    lemma_words = [token.lemma for token in doc.tokens]
    return lemma_words

def get_part_speech(doc, pos):
    part_speech = []
    for token in doc.tokens:
        if token.pos == pos:
            part_speech.append(token.text)
    return part_speech

def get_lemma_part_speech(doc, pos):
    lemma_part_speech = []
    for token in doc.tokens:
        if token.pos == pos:
            lemma_part_speech.append(token.lemma)
    return lemma_part_speech


In [102]:
get_lemma_part_speech(doc, 'NOUN')

['источник',
 'кристалл',
 'вычитка',
 'январь',
 'март',
 'глава',
 'пролог',
 'эпилог',
 'спб',
 'издательство',
 'кристалл',
 'илл',
 'камень',
 'чародей',
 'цвет',
 'пустынь',
 'площадей',
 'людей',
 'рассвет',
 'земля',
 'орел',
 'двуглавый',
 'лавр',
 'гигант',
 'скала',
 'забавой',
 'тиара',
 'колпак',
 'учитель',
 'мучитель',
 'властитель',
 'дурак',
 'снежок',
 'гоголек',
 'невнятный',
 'запутать',
 'первенец',
 'век',
 'взашей',
 'пыль',
 'мысль',
 'книга',
 'этой',
 'точка',
 'поток',
 'книга',
 'принцип',
 'прочтение',
 'игра',
 'заглавие',
 'книга',
 'свет',
 'контраст',
 'террорист',
 'философ',
 'контраст',
 'действительно',
 'установка',
 'принцип',
 'вещь',
 'взгляд',
 'роман',
 'скандал',
 'семейство',
 'половиной',
 'год',
 'дом',
 'жена',
 'музыкант',
 'муж',
 'чиновник',
 'сенатор',
 'обязанность',
 'департамент',
 'циркуляр',
 'слуга',
 'ночь',
 'сон',
 'сын',
 'студент',
 'отношение',
 'отец',
 'роман',
 'друг',
 'история',
 'любовь',
 'измена',
 'вражд

In [103]:
get_part_speech(doc, 'NOUN')

['Источник',
 'Кристалл',
 'вычитка',
 'января',
 'марта',
 'главах',
 'прологом',
 'эпилогом',
 'СПб',
 'Издательство',
 'Кристалл',
 'илл',
 'камни',
 'чародей',
 'цвета',
 'пустыни',
 'площадей',
 'людей',
 'рассвета',
 'земле',
 'орел',
 'двуглавый',
 'лаврах',
 'гигант',
 'скале',
 'забавой',
 'тиару',
 'колпак',
 'учитель',
 'мучитель',
 'властитель',
 'дурак',
 'снежок',
 'гоголек',
 'невнятен',
 'запутан',
 'первенец',
 'веком',
 'взашей',
 'пыль',
 'Мысль',
 'Книга',
 'этой',
 'точки',
 'потоком',
 'книги',
 'принцип',
 'прочтения',
 'игра',
 'заглавием',
 'книга',
 'свете',
 'контрастах',
 'террорист',
 'философ',
 'Контрасты',
 'действительно',
 'установка',
 'принцип',
 'вещи',
 'взгляд',
 'романа',
 'скандал',
 'семействе',
 'половиной',
 'года',
 'дома',
 'жена',
 'музыкантом',
 'Муж',
 'чиновник',
 'сенатор',
 'обязанности',
 'департамент',
 'циркуляры',
 'слугами',
 'ночам',
 'сны',
 'сына',
 'студента',
 'отношения',
 'отцом',
 'роман',
 'друга',
 'история',


In [70]:
get_lemma_words(doc)

['источник',
 ':',
 'андрей',
 'белый',
 ';',
 '"',
 'петербург',
 '"',
 ',',
 'роман',
 '.',
 'изд-во',
 ':',
 '"',
 'кристалл',
 '"',
 ',',
 'санкт-петербург',
 ',',
 '1999',
 '.',
 'ocr',
 'и',
 'вычитка',
 ':',
 'давид',
 'титиевский',
 '(',
 'davidtit',
 '@',
 'land',
 '.',
 'ru',
 ')',
 ',',
 'хайф',
 ',',
 '28',
 'январь',
 '2003',
 '.',
 'корректура',
 ':',
 'александр',
 'белоусенко',
 ',',
 '1',
 'март',
 '2003',
 '.',
 'роман',
 'в',
 'восемь',
 'глава',
 'с',
 'пролог',
 'и',
 'эпилог',
 'андрей',
 'белый',
 '.',
 'петербург',
 '/',
 'роман',
 '.',
 '--',
 'спб',
 '.',
 'ооо',
 '"',
 'издательство',
 '"',
 'кристалл',
 '"',
 '"',
 ',',
 '1999',
 '.',
 '--',
 '976',
 'с',
 '.',
 ',',
 'илл',
 '.',
 'только',
 'камень',
 'мы',
 'дать',
 'чародей',
 ',',
 'да',
 'нева',
 'буро-желтый',
 'цвет',
 ',',
 'да',
 'пустынь',
 'немой',
 'площадей',
 ',',
 'где',
 'казнить',
 'людей',
 'до',
 'рассвет',
 '.',
 'а',
 'что',
 'быть',
 'у',
 'мы',
 'на',
 'земля',
 ',',
 'чем',
